In [1]:
project = "gitops-project"

### Deploy Stream Processing

In [3]:
import v3io.dataplane

In [4]:
v3io_client = v3io.dataplane.Client()

In [5]:
v3io_client.stream.create(container='projects', stream_path=f'{project}/model-endpoints/stream', shard_count=1, raise_for_status=[204, 409])

In [6]:
import os

from mlrun import import_function
from mlrun.platforms import mount_v3io
from mlrun.runtimes import RemoteRuntime
import json

fn: RemoteRuntime = import_function("hub://model_monitoring_stream")

# Configures the nuclio trigger. See nuclio triggers https://nuclio.io/docs/latest/reference/triggers/.
fn.add_v3io_stream_trigger(
    stream_path=f"projects/{project}/model-endpoints/stream",
    name="monitoring_stream_trigger",
)

fn.set_env("MODEL_MONITORING_PARAMETERS", json.dumps(
    {"project": project, "v3io_framesd": os.environ.get("V3IO_FRAMESD")}))

fn.metadata.project = project
fn.apply(mount_v3io())

# Deploy the function
fn.deploy()

> 2021-07-30 23:46:52,157 [info] Starting remote function deploy
2021-07-30 23:46:52  (info) Deploying function
2021-07-30 23:46:52  (info) Building
2021-07-30 23:46:53  (info) Staging files and preparing base images
2021-07-30 23:46:53  (info) Building processor image
2021-07-30 23:46:55  (info) Build complete
2021-07-30 23:47:03  (info) Function deploy complete
> 2021-07-30 23:47:04,053 [info] function deployed, address=default-tenant.app.us-sales-eks.iguazio-cd0.com:30886


'http://default-tenant.app.us-sales-eks.iguazio-cd0.com:30886'

### Deploy Batch Processing

In [7]:
from mlrun import code_to_function, mount_v3io

In [8]:
fn = code_to_function(
    name="model-monitoring-batch",
    kind="job",
    image="mlrun/mlrun",
    filename="model_monitoring_batch.py",
    handler="handler",
)
fn.metadata.project = project
fn.apply(mount_v3io())

# Run the function as an MLRun job
fn.run(name='model-monitoring-batch', schedule="0 */1 * * *",
       params={"project": project})

> 2021-07-30 23:47:07,388 [info] starting run model-monitoring-batch uid=daf7ef66f5064699924ab6b3b984292d DB=http://mlrun-api:8080
> 2021-07-30 23:47:07,503 [info] task scheduled, {'schedule': '0 */1 * * *', 'project': 'gitops-project', 'name': 'model-monitoring-batch'}
